In [29]:
!python --version

Python 3.10.12


In [31]:
%%bash
set -e

# 1. Install Python 3.7 and its distutils (needed for get-pip.py)
apt-get update -qq
apt-get install -y python3.7 python3.7-distutils

# 2. Bootstrap pip under Python 3.7
curl -sS https://bootstrap.pypa.io/get-pip.py -o /tmp/get-pip.py
python3.7 /tmp/get-pip.py

# After this, you’ll have a “pip3.7” (and probably an unversioned “pip”) that points at Python 3.7.

# 3. Use update-alternatives to make “python3 → python3.7”
#    Give priority 2 to python3.7, lower priority 1 to the existing /usr/bin/python3 (3.11)
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2
update-alternatives --set python3 /usr/bin/python3.7

# 4. Use update-alternatives to make “pip → pip3.7”
#    First, register the old “pip” (from Python 3.11) with lower priority
if [ -x /usr/local/bin/pip ]; then
  update-alternatives --install /usr/bin/pip pip /usr/local/bin/pip 1
fi
#    Now register the new “pip3.7” as higher priority
if [ -x /usr/local/bin/pip3.7 ]; then
  update-alternatives --install /usr/bin/pip pip /usr/local/bin/pip3.7 2
  update-alternatives --set pip /usr/local/bin/pip3.7
fi

# 5. Verify that “python3” and “pip” now point at 3.7
echo "python → $(readlink -f "$(which python3)")"
python3 --version

echo "pip → $(readlink -f "$(which pip)")"
pip --version


Reading package lists...
Building dependency tree...
Reading state information...
python3-distutils is already the newest version (3.10.8-1~22.04).
python3.10 is already the newest version (3.10.12-1~22.04.9).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00

update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode
update-alternatives: using /usr/local/bin/pip to provide /usr/bin/pip (pip) in auto mode
update-alternatives: using /usr/local/bin/pip3.10 to provide /usr/bin/pip (pip) in auto mode
python → /usr/bin/python3.10
Python 3.10.12
pip → /usr/local/bin/pip
pip 25.1.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [32]:
!python --version

Python 3.10.12


In [33]:
!pip install cohere
!pip install faiss

  Using cached cohere-5.15.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached types_requests-2.32.0.20250602-py3-none-any.whl.metadata (2.1 kB)
Using cached cohere-5.15.0-py3-none-any.whl (259 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 119.2 MB/s eta 0:00:00
Using cached types_requests-2.32.0.20250602-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28/28 [cohere]
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [28]:
import faiss

ModuleNotFoundError: No module named 'faiss'

In [5]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import userdata

api_key = userdata.get("COHERE_API_KEY")
co = cohere.Client(api_key)

In [8]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""
texts = text.split(".")
texts = [t.strip().strip("\n") for t in texts]
texts


['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 'Interstellar uses extensive practical a

In [11]:
response = co.embed(texts=texts, input_type="search_document").embeddings
embeds = np.array(response)
embeds.shape

(15, 4096)